**Automatic optimization strategies**

In this code document, we will be experimenting with algorithms that can identify optimization routes in the production of ethanol in *Lactobacillus lactics*.

In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

In [2]:
model = read_sbml_model('iNF517.xml')

In [3]:
model

Name,iNF517
Memory address,0x01fa62c6acd0
Number of metabolites,650
Number of reactions,754
Number of groups,0
Objective expression,1.0*BIOMASS_LLA - 1.0*BIOMASS_LLA_reverse_0796e
Compartments,"cytosol, extracellular space"


In [4]:
model.reactions.EX_glc__D_e

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x01fa66047d60
Stoichiometry,glc__D_e <-- D-Glucose <--
GPR,
Lower bound,-2.12
Upper bound,-0.92


In [5]:
model.reactions.EX_etoh_e

Reaction identifier,EX_etoh_e
Name,Ethanol exchange
Memory address,0x01fa65f03bb0
Stoichiometry,etoh_e --> Ethanol -->
GPR,
Lower bound,0.44
Upper bound,1.02


**Ethanol yield calculations**

In [6]:
from cobra.flux_analysis import flux_variability_analysis

In [7]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.26
    model.medium = medium
    solution = model.optimize()
    
    # calculating max yield of etoh
    substrate_flux = model.reactions.EX_glc__D_e.flux
    model.objective = model.reactions.EX_etoh_e
    max_etoh_production = model.optimize().objective_value
    print("the flux of etoh production: ",max_etoh_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_etoh = max_etoh_production / (-1. * substrate_flux)
    print("The molar yield of 5etoh ", molar_yield_etoh, "[mmol-etoh / mmol-glc]")

the flux of etoh production:  1.02 [mmol gDW^-1 h^-1]
The molar yield of 5etoh  0.8095238095238095 [mmol-etoh / mmol-glc]


**Automatic optimization using OptGene**

The OptGene algorithm can be used to find genes which can be knocked out or mutated to increase the production of our target compound. The algorithm uses an approach, which takes a very long time to run. To make the program a tiny bit faster, all the essential reactions can be found and excluded from analysis, as they should not be knocked out in any case. Knocking out essential genes would stop growth.

In [ ]:
# In this block, we are finding a list of the essential reactions and the non-essential reactions. 
# By looping over all reactions in the model and seeing whether they can grow, we can sort them into the two different lists

#%%time

essential = []
non_essential = []

for reaction in model.reactions:
    with model:
        reaction.knock_out()
        if model.slim_optimize(error_value=0.) == 0.0:
            essential.append(reaction)
        else:
            non_essential.append(reaction)

In [ ]:
# Following this, we can find the genes associated with all of the essential reactions

#%%time
ess = []
for i in range (1,len(essential)):
    for gene in list(essential[i].genes):
        ess.append(gene)

In [ ]:
from cameo.strain_design import OptGene
optgene = OptGene(model,essential_genes=essential)
result = optgene.run(target=model.reactions.EX_etoh_e,
                     biomass=model.reactions.BIOMASS_LLA,
                     substrate=model.reactions.EX_glc__D_e,
                     max_evaluations=207,
                     plot=False)

In [ ]:
result

**Automatic optimization using OptKnock**

OptKnock uses a bi-level mixed integer linear programming approach to identify reaction knockouts

In [ ]:
from cameo.strain_design import OptKnock

In [ ]:
optknock = OptKnock(model, fraction_of_optimum=0.1)

Running multiple knockouts with OptKnock can take a few hours or days…

In [ ]:
result = optknock.run(max_knockouts=1, target="EX_etoh_e", biomass="BIOMASS_LLA")

**Knock out the P5CR had no significant effect on biomass and ethanol production**

In [ ]:
model.reactions.P5CR.genes

In [ ]:
# P5CR knockout
with model:    
    print("Normal growth conditions")
    print("Maximal biomass:")
    print(model.slim_optimize())
    model.objective = model.reactions.EX_etoh_e
    print("Maximal ethanol production rate:）")
    print('%.17f'%model.slim_optimize())
with model:
    model.genes.LLMG_RS10365.knock_out()
    print("\nWithout Pyrroline-5-carboxylate reductase")
    print("Maximal biomass:")
    print(model.slim_optimize())
    substrate_flux = model.reactions.EX_glc__D_e.flux
    model.objective = model.reactions.EX_etoh_e
    max_etoh_e_production = model.optimize().objective_value
    molar_yield_etoh_e = max_etoh_e_production / (-1. * substrate_flux)
    print("The molar yield of etoh_e ", molar_yield_etoh_e, "[mmol-etoh_e / mmol-glc]")
    print("Maximal ethanol production rate")
    print('%.17f'%model.slim_optimize())
    ppp_etoh_e = production_envelope(model,
                    reactions=[model.reactions.EX_etoh_e],
                    objective=model.reactions.BIOMASS_LLA)
    pppp = ppp_etoh_e.plot(x='EX_etoh_e', y='flux_maximum')
    fig = pppp.get_figure()
    plt.title(label="P5CR knockout")
    fig.savefig("Image/P5CR_knockout_ppp")

**Knock out (HIBD, SERD_L, IGPDH_1, FTHFCL_1) had no significant effect on biomass and ethanol production**

In [ ]:
model.reactions.HIBD.genes

In [ ]:
model.reactions.SERD_L.genes

In [ ]:
model.reactions.IGPDH_1.genes

In [ ]:
model.reactions.FTHFCL_1.genes

In [ ]:
# all knockout
with model:    
    print("Normal growth conditions")
    print("Maximal biomass:")
    print(model.slim_optimize())
    model.objective = model.reactions.EX_etoh_e
    print("Maximal ethanol production rate:）")
    print('%.17f'%model.slim_optimize())
with model:
    model.genes.LLMG_RS00960.knock_out()
    model.genes.LLMG_RS06560.knock_out()
    model.genes.LLMG_RS08705.knock_out()
    model.genes.LLMG_RS08710.knock_out()
    model.genes.LLMG_RS12530.knock_out()
    print("\nWithout all knockout")
    print("Maximal biomass:")
    print(model.slim_optimize())
    substrate_flux = model.reactions.EX_glc__D_e.flux
    model.objective = model.reactions.EX_etoh_e
    max_etoh_e_production = model.optimize().objective_value
    molar_yield_etoh_e = max_etoh_e_production / (-1. * substrate_flux)
    print("The molar yield of etoh_e ", molar_yield_etoh_e, "[mmol-etoh_e / mmol-glc]")
    print("Maximal ethanol production rate")
    print('%.17f'%model.slim_optimize())
    ppp_etoh_e = production_envelope(model,
                    reactions=[model.reactions.EX_etoh_e],
                    objective=model.reactions.BIOMASS_LLA)
    pppp = ppp_etoh_e.plot(x='EX_etoh_e', y='flux_maximum')
    fig = pppp.get_figure()
    plt.title(label="all knockout")
    fig.savefig("Image/all knockout_ppp")